In [1]:
!nvidia-smi

Tue Jan 30 22:11:11 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.40       Driver Version: 516.40       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P0    14W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\Daksh Patel\OneDrive\Desktop\SOP


In [3]:
!pip install ultralytics

In [4]:
from IPython import display
display.clear_output()

In [5]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.6 🚀 Python-3.11.7 torch-2.1.2+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Setup complete ✅ (8 CPUs, 7.8 GB RAM, 184.2/325.8 GB disk)


In [6]:
from ultralytics import YOLO
from IPython.display import display, Image

In [7]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="LgWWx0wIKsmwMMhgXefj")
project = rf.workspace("sop1").project("sop_arrhythmea_classification")
dataset = project.version(3).download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to SOP_arrhythmea_classification-3 in folder:: 100%|██████████| 2964/2964 [00:04<00:00, 674.81it/s]


In [9]:
## Splitting train data

def get_train_transforms():
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=0.5),
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
            ],p=0.9),
            A.ToGray(p=0.01),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=512, width=512, p=1),
            A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [10]:
## splitting to valid set

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [11]:
## Splitting to test set

def get_test_transforms():
    return A.Compose([
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [23]:
...
filenames = []
for filename in os.listdir(r"C:\Users\Daksh Patel\OneDrive\Desktop\SOP\SOP_arrhythmea_classification-3\_train\Normal"):
    filenames.append(filename)

In [24]:
filenames

['Normal-1-_png.rf.987ea5f83e947e7a1661547b95ea6ab7.jpg',
 'Normal-1-_png.rf.cd2b7d99ab1faf20110d8c5b05d2cf5c.jpg',
 'Normal-1-_png.rf.e8da9a3b596dbd232cd48f4a5d04596d.jpg',
 'Normal-10-_png.rf.1c575a2d2d116375e8336307c00b7d44.jpg',
 'Normal-10-_png.rf.657ed1d21015622f68113b78cc5aecd9.jpg',
 'Normal-10-_png.rf.ccfcc1cdc0d6a24409e7d9d665f665a4.jpg',
 'Normal-100-_png.rf.24636862ae6c664f7591cfb12417f9af.jpg',
 'Normal-100-_png.rf.6a435be2a6be43fa3f77fe5acdc7c9b2.jpg',
 'Normal-100-_png.rf.89c6cfa67fd71a9141e8fb37a447b60b.jpg',
 'Normal-101-_png.rf.010e6afe9cc400d8568c1a14b560e494.jpg',
 'Normal-101-_png.rf.7c58334db57e358e3fc51bf972ccc329.jpg',
 'Normal-101-_png.rf.b34f7a196862848ed937f73d3efd4bb0.jpg',
 'Normal-102-_png.rf.a42d4999d672d27ef724111c48fb0820.jpg',
 'Normal-102-_png.rf.f20c840bacb4af536188f6b6cffc4d70.jpg',
 'Normal-102-_png.rf.f4128c24ecfd1bd23743cae5484ab518.jpg',
 'Normal-104-_png.rf.06c4f18eb8e2e0d8dae85fa5a5776692.jpg',
 'Normal-104-_png.rf.14889a1381e77f9eecedad87a722

In [32]:
class DataAugmentation:
    """
    Handles with various augmentations for dataset.
    """

    def __init__(self):
        pass

    def fill(self, img, h, w):
        img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
        return img

    def horizontal_shift(self, img, ratio=0.0):
        if ratio > 1 or ratio < 0:
            print('Value should be less than 1 and greater than 0')
            return img
        ratio = random.uniform(-ratio, ratio)
        h, w = img.shape[:2]
        to_shift = w * ratio
        if ratio > 0:
            img = img[:, :int(w - to_shift), :]
        if ratio < 0:
            img = img[:, int(-1 * to_shift):, :]
        img = self.fill(img, h, w)
        return img

    def vertical_shift(self, img, ratio=0.0):
        if ratio > 1 or ratio < 0:
            print('Value should be less than 1 and greater than 0')
            return img
        ratio = random.uniform(-ratio, ratio)
        h, w = img.shape[:2]
        to_shift = h * ratio
        if ratio > 0:
            img = img[:int(h - to_shift), :, :]
        if ratio < 0:
            img = img[int(-1 * to_shift):, :, :]
        img = self.fill(img, h, w)
        return img

    def brightness(self, img, low, high):
        value = random.uniform(low, high)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        hsv = np.array(hsv, dtype=np.float64)
        hsv[:, :, 1] = hsv[:, :, 1] * value
        hsv[:, :, 1][hsv[:, :, 1] > 255] = 255
        hsv[:, :, 2] = hsv[:, :, 2] * value
        hsv[:, :, 2][hsv[:, :, 2] > 255] = 255
        hsv = np.array(hsv, dtype=np.uint8)
        img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        return img

    def channel_shift(self, img, value):
        value = int(random.uniform(-value, value))
        img = img + value
        img[:, :, :][img[:, :, :] > 255] = 255
        img[:, :, :][img[:, :, :] < 0] = 0
        img = img.astype(np.uint8)
        return img

    def zoom(self, img, value):
        if value > 1 or value < 0:
            print('Value for zoom should be less than 1 and greater than 0')
            return img
        value = random.uniform(value, 1)
        h, w = img.shape[:2]
        h_taken = int(value * h)
        w_taken = int(value * w)
        h_start = random.randint(0, h - h_taken)
        w_start = random.randint(0, w - w_taken)
        img = img[h_start:h_start + h_taken, w_start:w_start + w_taken, :]
        img = self.fill(img, h, w)
        return img

    def horizontal_flip(self, img, flag):
        if flag:
            return cv2.flip(img, 1)
        else:
            return img

    def vertical_flip(self, img, flag):
        if flag:
            return cv2.flip(img, 0)
        else:
            return img

    def rotation(self, img, angle):
        angle = int(random.uniform(-angle, angle))
        h, w = img.shape[:2]
        M = cv2.getRotationMatrix2D((int(w / 2), int(h / 2)), angle, 1)
        img = cv2.warpAffine(img, M, (w, h))
        return img

    def process(self, dataset_directory, augmentation_dataset_directory, cl):
        if not os.path.exists(os.path.join(augmentation_dataset_directory, cl)):
            os.mkdir(os.path.join(augmentation_dataset_directory, cl))

        for each_file in os.listdir(os.path.join(dataset_directory, cl)):
            filename, file_extension = os.path.splitext(each_file)

            if file_extension in ['.jpg', '.jpeg', '.png']:
                image = cv2.imread(os.path.join(dataset_directory, cl, each_file))
                multi_images = (
                    self.horizontal_shift(image), self.vertical_shift(image), self.brightness(image, 0.5, 3),
                    self.zoom(image, 0.5), self.channel_shift(image, 60), self.horizontal_flip(image, True),
                    self.vertical_flip(image, True), self.rotation(image, 60))

                _file_name = 0
                for each_element in multi_images:
                    image = each_element
                    cv2.imwrite(
                        os.path.join(augmentation_dataset_directory, cl, f"{each_file[:-4]}" + "_" + f"{_file_name}" + ".jpg"),
                        image)
                    _file_name += 1

